In [ ]:
import torch
import torch as ch
from os import path
import numpy as np
from torch.utils.data import DataLoader
from prediction.outcome_prediction.Transformer.lightning_wrapper import LitModel
from prediction.short_term_outcome_prediction.timeseries_decomposition import BucketBatchSampler, \
    prepare_subsequence_dataset

In [ ]:
data_splits_path = '/Users/jk1/Downloads/train_data_splits_early_neurological_deterioration_ts0.8_rs42_ns5.pth'

In [ ]:
use_gpu = False
batch_size = 32

In [ ]:
splits = ch.load(path.join(data_splits_path))

In [ ]:
target_variable = 'max_NIHSS'
target_variable_index = np.where(splits[0][0][0][0][:, 2] == target_variable)[0][0]
target_variable_index

In [ ]:
all_datasets = [prepare_subsequence_dataset(x, use_gpu=use_gpu, use_target_timeseries=True, 
                                                target_timeseries_length=1) for x in splits]

In [ ]:
splits[0][0].shape

In [ ]:
train_dataset, val_dataset = all_datasets[0]

In [ ]:
len(train_dataset), len(val_dataset)

In [ ]:
train_dataset[2][0].shape, train_dataset[2][2].shape 


In [ ]:
train_bucket_sampler = BucketBatchSampler(train_dataset.idx_to_len_map, batch_size)
train_loader = DataLoader(train_dataset, batch_sampler=train_bucket_sampler,
                          # shuffling is done in the bucket sampler
                          shuffle=False, drop_last=False)

In [ ]:
# get first item from train_loader
for x in train_loader:
    print(x[0].shape, x[1].shape, x[2].shape)
    break

In [ ]:
model = ch.nn.Transformer(d_model=84, nhead=2, batch_first=True)

In [ ]:
module = LitModel(model, lr=1e-3, wd=1e-5, train_noise=0.1)

In [ ]:
train_x = ch.randn(32, 5, 10)
train_y = ch.randn(32, 2, 10)

In [ ]:
train_y.shape

In [ ]:
res = model(train_x, train_y)
res.shape

In [ ]:
x[2].shape

In [ ]:
# vector with torch infinite values of shape x[2].shape
torch.full(x[2].shape, float('inf'))

In [ ]:
res = model(x[0], torch.full(x[2].shape, float('inf')))
res.shape

In [ ]:
res

In [ ]:
test = ch.nn.Linear(10, 5)

In [ ]:
test(ch.randn(32, 5, 10)).shape

In [ ]:
x[2][0].shape

In [ ]:
import torchmetrics
from torchmetrics.regression import CosineSimilarity
metric = CosineSimilarity()
torchmetrics.functional.pairwise_cosine_similarity(x[2][0], res[0]), metric(x[2][0], res[0])